# What is stixdcpy
stixdcpy is a python package that facilitates access and analysis of STIX data. It provides APIs to query and download data from STIX data center (https://datacenter.stix.i4ds.net/) and s a set of tools for visualizing data and performing common analysis tasks.  With stixdcpy, users can query and download the following types of data from STIX data center:

- Quick-look light curves
- Housekeeping data
- Science data
- Energy calibration data
- Auxilary data
- STIX solar flare list



# Installation
```sh 
pip install stixdcpy
```
stixdcpy can be also installed from the stixdcpy github repository:
```sh 
pip install git+https://github.com/drhlxiao/stixdcpy.git
```

# Tutorial and documentation

- [Tutorial on Google Colab](https://colab.research.google.com/drive/17fQfbWjL0s0TpblbPL1Ysy_zFXj40FBf?usp=sharing)
- [Documentation](https://drhlxiao.github.io/stixdcpy/)

# Reporting Issues and Contributing

Open an issue on GitHub to report a problem. Pull requests welcome.

https://github.com/drhlxiao/stixdcpy





# Install dependencies 

In [1]:
#Install stixdcpy from pypi
#!pip install stixdcpy
# Or install from stixdcpy github repository
!pip install git+https://github.com/drhlxiao/stixdcpy.git
!pip install numpy matplotlib pandas astropy joblib

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/drhlxiao/stixdcpy.git to /tmp/pip-req-build-ixunsniz
  Running command git clone --filter=blob:none --quiet https://github.com/drhlxiao/stixdcpy.git /tmp/pip-req-build-ixunsniz
  Resolved https://github.com/drhlxiao/stixdcpy.git to commit 5e1c3051fcfcba4b09ccfb1b567d633ebd3d49ff
  Preparing metadata (setup.py) ... done
  Created wheel for stixdcpy: filename=stixdcpy-2.1-py3-none-any.whl size=46634 sha256=8b03e4b24f1486c889cd0499ca3f51e2568006ebcf9240a35d4755d7ebca3b72
  Stored in directory: /tmp/pip-ephem-wheel-cache-5soorj5k/wheels/b9/23/8d/a9b62c15959ccd15d3fe010cb0e8605adec4c658e71dbfa0dc
Successfully built stixdcpy
  Attempting uninstall: stixdcpy
    Found existing installation: stixdcpy 1.0.2
    Uninstalling stixdcpy-1.0.2:
      Successfully uninstalled stixdcpy-1.0.2
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
from pprint import pprint
from datetime import timedelta
from matplotlib import pyplot as plt
from astropy.table import QTable
from stixdcpy.quicklook import LightCurves
from stixdcpy.energylut import EnergyLUT
from stixdcpy import auxiliary as aux
from stixdcpy.net import FitsQuery as fq
from stixdcpy.net import JSONRequest as jreq
from stixdcpy import instrument as inst
from stixdcpy.science import ScienceL1, Spectrogram
from stixdcpy.housekeeping import Housekeeping
from stixdcpy import detector_view as dv
from google.colab import data_table
data_table.enable_dataframe_formatter()
%matplotlib inline

ModuleNotFoundError: No module named 'google.colab'

# Request Quick-look  data from STIX data center 

Fetch light curves from STIX data center

In [ ]:
lc=LightCurves.from_sdc(start_utc='2022-01-01T05:00:00', end_utc='2022-01-01T10:00:00', ltc=True)

where: 
* start_utc -  start time (onboard time); 
* end_utc - end time ; 
* ltc - an optional argument.  Perform light time correction to light curves if it is True

It returns a data object



Preview the QL lightcurves

In [ ]:
lc.peek()

Access the data:

In [ ]:
lc.data.keys()
lc.data['counts']

# Housekeep data

Fetch housekeeping data from STIX data center:



In [ ]:
hk=Housekeeping.from_sdc(start_utc='2021-09-06T12:00:00', end_utc='2021-09-06T14:00:00')
pprint(hk.param_names)


Plot housekeeping data time series:

In [ ]:
hk.plot('NIX00078,NIX00079,NIX00080,NIX00081')


You need to provide the names of parameters to be plotted.

One could also plot time series of parameters using plt

In [ ]:
plt.step( hk.data['datetime'], hk.data['raw_values']['NIXD0023'])
plt.xlabel('UTC')
plt.ylabel('Mode')

# Calibration data

Request calibration data from STIX data center

In [ ]:
elut=EnergyLUT.request('2021-09-03T00:00:00')

Print information on the onboard elut and the calibration run peformed closest to the specified time:

In [ ]:
elut.info()


Print energy calibration factors 

In [ ]:
elut.data

Print the real energy range of bins of a pixel

In [ ]:
lut=elut.get_pixel_true_ebins(pixel=34)

pd.DataFrame(lut)

# S/C Ephemeris 

Fetch s/c ephemeris data from STIX data center: 

In [ ]:
emph=aux.Ephemeris.from_sdc(start_utc='2022-01-01T00:00:00', end_utc='2022-03-31T01:00:00', steps=100)

Preview the data

In [ ]:
emph.peek()

In [ ]:
pprint(emph.data['orbit'])

# STIX pointing

In [ ]:
fov=aux.Pointing.from_sdc('2022-03-19T00:00:00')
fov.peek()


The web tool at the link https://datacenter.stix.i4ds.net/view/ancillary provides more detailed information of STIX pointing

In [ ]:
#Let's plot the data
pprint(fov.data)

# Solar flare light time difference and viewing angle calculator

This tool allows calculation of the light time difference between a flare and observers at earth or solar orbiter, as well as the viewing angles. 

In [ ]:
flare_aux=jreq.request_flare_light_time_and_angle(utc='2022-03-01T00:00:00', flare_x=100, flare_y=1000, observer='stix')


pprint(flare_aux)

where:        

* utc -    observation time
* flare_x -  flare helio-projective longitude in units of arcsec as seen by the observer 
* flare_y -  flare helio-projective latitude in units of arcsec 
* observer: It can be either "earth" or "stix" . The default value is "earth". 

A data object is returnned


The tool is available at the link
https://datacenter.stix.i4ds.net/view/ancillary


# Access and preview science data

## level-1 science data


Let's download a level-1 science data file from STIX data center

In [ ]:
l1_obj=ScienceL1.from_sdc(request_id=2112240046)

Here **2112240046** is the requests ID (also called unique id), which can  can be found on the science data web page https://datacenter.stix.i4ds.net/view/list/bsd.   
**ltc** is an optional argument. Light time correction will be performed if ltc = True. 
**l1_obj** is a data object 

Access the FITS Header Data Unit List attached to the data object:

In [ ]:
hdul=l1_obj.hdul
hdul.info()

Note that FITS hdul is only one of the data attributes.  

Let's print the hdul header

In [ ]:
hdul['PRIMARY'].header

Preview science data 

In [ ]:
l1_obj.peek('spg,lc, spec,tbin, qllc')


Peform dead time correction to the data

In [ ]:
corrected_data=l1_obj.correct_dead_time()

Print the array dimensions

In [ ]:
corrected_data.keys()
for key, value in corrected_data.items():
    print(key, value.shape)

Plot the live ratios of detectors: 

In [ ]:
plt.figure()
plt.plot(l1_obj.time, corrected_data['live_ratio'])
plt.xlabel('Time (s)')
plt.ylabel('Live ratio of detector groups')

Plot pixel and energy integerated corrected count rate:

In [ ]:
plt.figure()
plt.plot(l1_obj.time, np.sum(corrected_data['corrected_rates'], axis=(2,3) ))


Visualize counts with the module DetectorView

In [ ]:
pixel_counts = l1_obj.pixel_total_counts
det_view = dv.DetectorView(pixel_counts)
det_view.display()

In [ ]:
l1_obj.ebins_high

In [ ]:
#print(l1_obj.url)
#link to science browser 

## Download and plot spectrograms

Let's fetch a spectrogram data file from STIX data center

In [ ]:
request_id=	2108150003
l4d=Spectrogram.from_sdc(request_id=request_id)


In [ ]:
l4d.hdul.info()
l4d.peek()

# Search and download FITS files from STIX data center

Let's set download location first: 

In [ ]:
fq.chdir('./downloads')



In [ ]:
res=fq.query('2022-02-10T05:00:00', '2022-02-18T07:00:00', product_type='xray-cpd',filter='BKG')


Arguments: 
* **product_type**   options:  
| option        	| Data type             	|
|--------------	|----------------------	|
| any      	| Any type (default value)          	|
| science      	| Any science          	|
| quicklook    	| Any quick-look       	|
| housekeeping 	| Housekeeping         	|
| xray-rpd     	| Raw pixel data       	|
| xray-cpd     	| Pixel data           	|
| xray-scpd    	| Summed pixel data    	|
| xray-vis     	| Visibility           	|
| xray-spec    	| Spectrogram          	|
| ql-lc        	| QL light curve       	|
| ql-var       	| QL variance          	|
| ql-bkg       	| QL background        	|
| ql-spec      	| QL spectra           	|
| ql-ffl       	| QL flare location    	|
| ql-cal       	| Calibration spectrum 	|
| aspect       	| Bulk aspect          	|
| auxiliary    	| Auxiliary data       	|


* filter: optional. 
    - Use to filter FITS files. Only science data with descriptions containing the specified string will be selected. 
    set it to None if you  don't want to filter the results

An instance of FitQueryResult  is returned. 



Convert the query result to pandas dataframe: 


In [ ]:
df=res.to_pandas()
df


One could download all the FITS file from stix data center by using the function "fetch": 

In [ ]:
fits_list=res.fetch()

Note that if a FITS file is found on the local disk, it will not be downloaded again from STIX data center. 

Open all the download FITS file: 

In [ ]:
hduls=res.open_fits()


hduls is a list of hdu objects

Print file info

In [ ]:
res.fits_info()

# Download flare list from STIX data center

In [ ]:
flares=jreq.fetch_flare_list('2021-04-08T00:00:00',  '2021-05-09T00:00:00', sort='goes' )

#the "flares" is a python list
pprint(flares)


Parameters:

* start_UTC: str
  - start time
* end_UTC: str
  - end time

* sorting key: str, options:
 
- LC0, flare peak counts of the 4 - 10 keV light curve 

- LC1, flare peak counts of 10 - 10 keV light curves
- LC2, similar to above, but of the third energy band
- LC3,  similar to above, but of the fourth energy band
- LC4,  similar to above, but of the fifth  energy band
- time,  flare observation time
- goes, goes class 

Display the list in  table view:


In [ ]:
df=pd.DataFrame(flares)
df